In [1]:
# Dependencies
import pandas as pd
import json
import requests
import time
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
from pprint import pprint

In [55]:
# Import Firt Data Source: Dog Names CSV, and format
dog_names_file = "resources/dog_names.csv"
dog_names_df = pd.read_csv(dog_names_file)

dog_names_df_clean = dog_names_df.rename(columns={"Row_Labels": "Name",
                                                  "Count_AnimalName": "Count"})

dog_names_df_clean['Name'] = dog_names_df_clean.Name.str.title()
dog_names_df_clean.set_index("Name", inplace=True)

dog_names_df_clean.head(10)

,Count
Name,
1,1
2,2
40804,1
90201,1
90203,1
102201,1
3010271,1
March,2
April,51


In [56]:
# Import second source of Data: State Names CSV
state_names_file = "resources/state_names.csv"
state_names_df = pd.read_csv(state_names_file)

state_names_df.set_index("Id", inplace=True)
state_names_df.set_index("Name", inplace=True)

state_names_df.head()

,Year,Gender,State,Count
Name,,,,
Mary,1910.0,F,AK,14.0
Annie,1910.0,F,AK,12.0
Anna,1910.0,F,AK,10.0
Margaret,1910.0,F,AK,8.0
Helen,1910.0,F,AK,7.0


In [57]:
# Filter data by year: 2011 or greater
state_names_2000_df = state_names_df.loc[state_names_df['Year'] >= 2011]
state_names_2000_df.head()

,Year,Gender,State,Count
Name,,,,
Olivia,2011.0,F,AK,60.0
Emma,2011.0,F,AK,56.0
Isabella,2011.0,F,AK,50.0
Madison,2011.0,F,AK,46.0
Sophia,2011.0,F,AK,44.0


In [58]:
# Filter Data by State: NY
state_names_2000_NY_df = state_names_2000_df.loc[state_names_2000_df["State"] == "NY"]
state_names_2000_NY_df.head()

,Year,Gender,State,Count
Name,,,,
Sophia,2011.0,F,NY,1453.0
Isabella,2011.0,F,NY,1415.0
Olivia,2011.0,F,NY,1175.0
Emma,2011.0,F,NY,1122.0
Ava,2011.0,F,NY,954.0


In [59]:
# Filter data by top 100 Names from NY
state_names_2000_NY_100_df = state_names_2000_NY_df.iloc[:100]
state_names_2000_NY_100_df.head()

,Year,Gender,State,Count
Name,,,,
Sophia,2011.0,F,NY,1453.0
Isabella,2011.0,F,NY,1415.0
Olivia,2011.0,F,NY,1175.0
Emma,2011.0,F,NY,1122.0
Ava,2011.0,F,NY,954.0


In [42]:
# Third source of data: Name Origins API 

# Connection:

# API URLs & Key
origin_url = f"https://www.behindthename.com/api/lookup.json"
related_url = f"https://www.behindthename.com/api/related.json"
api_key = "&key=an594138819"

# Test Example:
query_name = "Emily"
query_name_origin = origin_url + "?name=" + query_name + api_key
query_name_related = related_url + "?name=" + query_name + api_key

In [43]:
# Test response for origin_url
origin_data = requests.get(query_name_origin).json()
pprint(origin_data)

[{'gender': 'f',
  'name': 'Emily',
  'usages': [{'usage_code': 'eng',
              'usage_full': 'English',
              'usage_gender': 'f'}]}]


In [44]:
# Test response for related_url
related_data = requests.get(query_name_related).json()
pprint(related_data)

{'names': ['Amilia',
           'Em',
           'Emalee',
           'Emely',
           'Emilee',
           'Emilia',
           'Emmie',
           'Emmy',
           'Millie',
           'Milly']}


In [38]:
# Create a Name list from top 100 names:
name_list = []

for name in state_names_2000_NY_100_df["Name"]:
    name_list.append(name)
print(name_list)

['Sophia', 'Isabella', 'Olivia', 'Emma', 'Ava', 'Emily', 'Mia', 'Madison', 'Abigail', 'Chloe', 'Leah', 'Ella', 'Sofia', 'Gabriella', 'Grace', 'Sarah', 'Samantha', 'Gianna', 'Victoria', 'Elizabeth', 'Hailey', 'Ashley', 'Lily', 'Charlotte', 'Brianna', 'Julia', 'Natalie', 'Zoe', 'Kayla', 'Amelia', 'Hannah', 'Arianna', 'Rachel', 'Angelina', 'Esther', 'Maya', 'Alexandra', 'Alyssa', 'Kaylee', 'Alexa', 'Anna', 'Allison', 'Addison', 'Avery', 'Aaliyah', 'Savannah', 'Alexis', 'Lillian', 'Evelyn', 'Layla', 'Camila', 'Katherine', 'Taylor', 'Sara', 'Riley', 'Nevaeh', 'Nicole', 'Makayla', 'Eva', 'Zoey', 'Peyton', 'Brooke', 'Ariana', 'Khloe', 'Sophie', 'Aubrey', 'Bella', 'Kylie', 'Chaya', 'Claire', 'Lauren', 'Adriana', 'Madeline', 'Melanie', 'Jasmine', 'Molly', 'Valentina', 'Kimberly', 'Kaitlyn', 'Maria', 'Michelle', 'Juliana', 'Mackenzie', 'Madelyn', 'Miriam', 'Scarlett', 'Genesis', 'Julianna', 'Stella', 'London', 'Natalia', 'Sydney', 'Serenity', 'Faith', 'Gabrielle', 'Jessica', 'Morgan', 'Naomi', '

In [45]:
# For loop call to API to retrieve origins from 'name_list':
genders = []
origins = []
origins_two = []
related_names = []

name_data=[]
for name in name_list:
    origin_query = origin_url + "?name=" + name + api_key
    related_query = related_url + "?name=" + name + api_key
    print(origin_query)
    print(related_query)
    print("----------------------------")
    response = requests.get(origin_query).json()
    response2 = requests.get(related_query).json()
    print(response[0]['gender'])
    print(response[0]['usages'][0]['usage_full'])
#     print(response[0]['usages'][1]['usage_full'])
    print(response2['names'][0:2])
    genders.append(response[0]['gender'])
    origins.append(response[0]['usages'][0]['usage_full'])
#     origins_two.append(response[0]['usages'][1]['usage_full'])
    related_names.append(response2['names'][0:2])


https://www.behindthename.com/api/lookup.json?name=Sophia&key=an594138819
https://www.behindthename.com/api/related.json?name=Sophia&key=an594138819
----------------------------
f
English
['Sofia', 'Sophy']
https://www.behindthename.com/api/lookup.json?name=Isabella&key=an594138819
https://www.behindthename.com/api/related.json?name=Isabella&key=an594138819
----------------------------
f
Italian
['Bella', 'Belle']
https://www.behindthename.com/api/lookup.json?name=Olivia&key=an594138819
https://www.behindthename.com/api/related.json?name=Olivia&key=an594138819
----------------------------
f
English
['Alivia', 'Liv']
https://www.behindthename.com/api/lookup.json?name=Emma&key=an594138819
https://www.behindthename.com/api/related.json?name=Emma&key=an594138819
----------------------------
f
English
['Em', 'Ema']
https://www.behindthename.com/api/lookup.json?name=Ava&key=an594138819
https://www.behindthename.com/api/related.json?name=Ava&key=an594138819
----------------------------
f
Engl

f
English
['Alex', 'Alexandra']
https://www.behindthename.com/api/lookup.json?name=Anna&key=an594138819
https://www.behindthename.com/api/related.json?name=Anna&key=an594138819
----------------------------
f
English
['Aina', 'Ana']
https://www.behindthename.com/api/lookup.json?name=Allison&key=an594138819
https://www.behindthename.com/api/related.json?name=Allison&key=an594138819
----------------------------
f
English
['Adelaide', 'Alease']
https://www.behindthename.com/api/lookup.json?name=Addison&key=an594138819
https://www.behindthename.com/api/related.json?name=Addison&key=an594138819
----------------------------
fm
English
['Addyson']
https://www.behindthename.com/api/lookup.json?name=Avery&key=an594138819
https://www.behindthename.com/api/related.json?name=Avery&key=an594138819
----------------------------
mf
English
[]
https://www.behindthename.com/api/lookup.json?name=Aaliyah&key=an594138819
https://www.behindthename.com/api/related.json?name=Aaliyah&key=an594138819
-----------

f
French
['Chelle', 'Micha']
https://www.behindthename.com/api/lookup.json?name=Juliana&key=an594138819
https://www.behindthename.com/api/related.json?name=Juliana&key=an594138819
----------------------------
f
Dutch
['Iuliana', 'Juliane']
https://www.behindthename.com/api/lookup.json?name=Mackenzie&key=an594138819
https://www.behindthename.com/api/related.json?name=Mackenzie&key=an594138819
----------------------------
fm
English
['Makenzie', 'Mckenzie']
https://www.behindthename.com/api/lookup.json?name=Madelyn&key=an594138819
https://www.behindthename.com/api/related.json?name=Madelyn&key=an594138819
----------------------------
f
English
['Madalyn', 'Maddie']
https://www.behindthename.com/api/lookup.json?name=Miriam&key=an594138819
https://www.behindthename.com/api/related.json?name=Miriam&key=an594138819
----------------------------
f
Hebrew
['Mirjam']
https://www.behindthename.com/api/lookup.json?name=Scarlett&key=an594138819
https://www.behindthename.com/api/related.json?name=Sc

In [47]:
# Lists retrieved by for loop:
print(origins)
print(genders)
print(related_names)

['English', 'Italian', 'English', 'English', 'English', 'English', 'Swedish', 'English', 'English', 'English', 'English', 'English', 'Norwegian', 'Italian', 'English', 'English', 'English', 'Italian', 'English', 'English', 'English (Modern)', 'English', 'English', 'French', 'English', 'English', 'English', 'English', 'English', 'English', 'English', 'Italian', 'English', 'Italian', 'English', 'Hinduism', 'English', 'English', 'English (Modern)', 'English', 'English', 'English', 'English', 'English', 'Arabic', 'English', 'German', 'English', 'English', 'Arabic', 'Spanish', 'English', 'English', 'Greek', 'English', 'English (Modern)', 'French', 'English (Modern)', 'Spanish', 'English (Modern)', 'English', 'English', 'Portuguese', 'English (Modern)', 'French', 'English', 'English', 'English', 'Hebrew', 'French', 'English', 'Italian', 'English', 'English', 'English', 'English', 'Italian', 'English', 'English (Modern)', 'Italian', 'French', 'Dutch', 'English', 'English', 'Hebrew', 'English'

In [61]:
# Create Data frame for data retrieved from API:
name_origins_dict = {
    'Name': name_list,
    'Genders' : genders, 
    'Origins' : origins,
    'Related Names' : related_names
}

name_origins_df = pd.DataFrame.from_dict(name_origins_dict)
name_origins_df.set_index("Name", inplace=True)

name_origins_df

,Genders,Origins,Related Names
Name,,,
Sophia,f,English,"[Sofia, Sophy]"
Isabella,f,Italian,"[Bella, Belle]"
Olivia,f,English,"[Alivia, Liv]"
Emma,f,English,"[Em, Ema]"
Ava,f,English,"[Eva, Avelina]"
Emily,f,English,"[Amilia, Em]"
Mia,f,Swedish,"[Jet, Jette]"
Madison,fm,English,"[Maddie, Maddison]"
Abigail,f,English,"[Abbey, Abbi]"


In [ ]:
# SQL DataBase connection:
connection_string = "root:Ag8998999!@localhost/Dog_vs_Human_Names_db"
engine = create_engine(f'mysql://{connection_string}')

In [ ]:
# Connection Test:
engine.table_names()

In [ ]:
# Export dog name data to SQL:
dog_names_df_clean.to_sql(name='dog_names', con=engine, if_exists='append', index=True)

In [ ]:
# Export human name data to SQL:
state_names_2000_NY_df.to_sql(name='state_names', con=engine, if_exists='append', index=True)

In [ ]:
# Export name origin data to SQL: